In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#import
import pandas as pd
import numpy as np
import tensorflow as tf
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split
from sklearn.manifold import TSNE, Isomap
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, Normalizer
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import uniform, randint
import xgboost as xgb
from sklearn.metrics import auc, accuracy_score, mean_absolute_error
from sklearn.model_selection import cross_val_score, RandomizedSearchCV, train_test_split, KFold
from sklearn.decomposition import PCA
from sklearn.multioutput import MultiOutputRegressor
import random
import copy

In [ ]:
# %cd /content/drive/MyDrive/Colab Notebooks/BigData_proj/model

df_old =  pd.read_csv('/content/drive/MyDrive/Colab Notebooks/fifa/fifa_dataset_without_value.csv')

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/fifa/fifa_dataset_without_value.csv')
#date 형식 변환
#df['date'] = [i.replace('-','') for i in df['date']]
#all_list : tournament,home_team, away_team
all_list = ((df['tournament'].unique()).tolist() + (df['home_team'].unique()).tolist() + (df['away_team'].unique()).tolist())

uni_len = '***'.join(all_list)
nat_enc = tf.keras.preprocessing.text.Tokenizer(num_words=len(all_list),
    filters='',
    lower=False,
    split = '***')
nat_enc.fit_on_texts([uni_len])
nat_dic = nat_enc.word_index

In [ ]:
#값을 enumerate로 변환
for step, val in enumerate(df['tournament']):
    df['tournament'].iloc[step] = (nat_dic.get(val))
for step, val in enumerate(df['home_team']):
    df['home_team'].iloc[step] = (nat_dic.get(val))
for step, val in enumerate(df['away_team']):
    df['away_team'].iloc[step] = (nat_dic.get(val))
for step, val in enumerate(df['neutral']):
    if df['neutral'].iloc[step] == False:
        df['neutral'].iloc[step] = 0
    if df['neutral'].iloc[step] == True:
        df['neutral'].iloc[step] = 1

In [ ]:
df.head()

,Unnamed: 0,home_team,away_team,home_score,away_score,tournament,neutral,year,home_team_total_points,away_team_total_points,...,away_M1_point,away_M2_point,away_M3_point,away_M4_point,away_T1_point,away_T2_point,away_K1_point,home_B1_value,home_B2_value,home_B3_value
0,36117,1,10,2.0,1.0,91,1,2014,603.0,656.0,...,78.0,77.0,75.0,75.0,80.0,78.0,80.0,1400000.0,2000000.0,1300000.0
1,36121,2,58,1.0,0.0,91,1,2014,714.0,529.0,...,72.0,69.0,67.0,63.0,73.0,72.0,66.0,2000000.0,2500000.0,950000.0
2,36127,3,1,1.0,1.0,91,0,2014,579.0,603.0,...,77.0,76.0,72.0,71.0,76.0,74.0,70.0,2500000.0,1500000.0,975000.0
3,36135,4,25,0.0,3.0,92,1,2014,500.0,680.0,...,79.0,78.0,76.0,75.0,87.0,82.0,80.0,1500000.0,3500000.0,2400000.0
4,36137,3,10,1.0,3.0,91,0,2014,579.0,656.0,...,78.0,77.0,75.0,75.0,80.0,78.0,80.0,2500000.0,1500000.0,975000.0


In [ ]:
#df_new
df_new = df
#len
temp_tor = []
for i in df_old['tournament']:
    temp_tor.append(len(i))
temp_away = []
for i in df_old['away_team']:
    temp_away.append(len(i))
temp_home = []
for i in df_old['home_team']:
    temp_home.append(len(i))
df_new['tor_len'] = temp_tor
df_new['home_len'] = temp_home
df_new['away_len'] = temp_away

# 결측값 있는 행 제거
df_new = df_new.dropna()

df_label1 = df_new['home_score']
df_label2 = df_new['away_score']

In [ ]:
#df_new, df_label 분리 : 모델의 인풋(df_new)과 아웃풋(df_label, home_score 과 away score)
df_new = df_new.drop(['home_score', 'away_score'], axis = 1)
#df_label
df_label = pd.concat([df_label1, df_label2],axis = 1)
df_label.columns = ['home_score', 'away_score']

**df_label** : 왜 인덱스가 7948까지 있을까? 표시는 6547 rows라고 되긴 함

In [ ]:
# concat이 단순하게 합치는 연산이라 index 갱신이 안된듯 

In [ ]:
df_label

,home_score,away_score
0,2.0,1.0
1,1.0,0.0
2,1.0,1.0
3,0.0,3.0
4,1.0,3.0
...,...,...
2543,0.0,0.0
2544,0.0,5.0
2545,1.0,1.0
2546,0.0,0.0


In [ ]:
x_train = df_new.to_numpy()
df_train = pd.DataFrame(data=x_train, columns=df_new.columns)

**df_train**

In [ ]:
df_train

,Unnamed: 0,home_team,away_team,tournament,neutral,year,home_team_total_points,away_team_total_points,home_B1_point,home_B2_point,...,away_M4_point,away_T1_point,away_T2_point,away_K1_point,home_B1_value,home_B2_value,home_B3_value,tor_len,home_len,away_len
0,36117,1,10,91,1,2014,603.0,656.0,73.0,72.0,...,75.0,80.0,78.0,80.0,1400000.0,2000000.0,1300000.0,28,4,7
1,36121,2,58,91,1,2014,714.0,529.0,73.0,72.0,...,63.0,73.0,72.0,66.0,2000000.0,2500000.0,950000.0,28,5,5
2,36127,3,1,91,0,2014,579.0,603.0,72.0,72.0,...,71.0,76.0,74.0,70.0,2500000.0,1500000.0,975000.0,28,12,4
3,36135,4,25,92,1,2014,500.0,680.0,74.0,74.0,...,75.0,87.0,82.0,80.0,1500000.0,3500000.0,2400000.0,8,6,6
4,36137,3,10,91,0,2014,579.0,656.0,72.0,72.0,...,75.0,80.0,78.0,80.0,2500000.0,1500000.0,975000.0,28,12,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2543,44145,85,16,92,0,2022,1243.8,1335.36,67.0,66.0,...,67.0,72.0,67.0,66.0,900000.0,900000.0,1200000.0,8,10,8
2544,44147,82,27,92,0,2022,961.23,1434.68,61.0,58.0,...,75.0,78.0,75.0,76.0,500000.0,300000.0,300000.0,8,7,7
2545,44150,4,34,92,0,2022,1473.04,1405.6,77.0,77.0,...,72.0,75.0,73.0,83.0,9500000.0,8000000.0,9000000.0,8,6,7
2546,44152,55,6,92,0,2022,1425.59,1509.61,77.0,75.0,...,76.0,79.0,77.0,78.0,7500000.0,3500000.0,4000000.0,8,8,5


In [ ]:
#x_train = df_new.to_numpy()
Norm = Normalizer() # 각 변수의 값을 원점으로부터 1만큼 떨어져 있는 값으로 변환 
x_temp = Norm.fit_transform(x_train)
df_temp = pd.DataFrame(data=x_temp, columns=df_new.columns)

**df_temp** : df_train은 x_train을 사용, df_temp는 x_train을 normalize한 값인 x_temp 사용

In [ ]:
df_temp

,Unnamed: 0,home_team,away_team,tournament,neutral,year,home_team_total_points,away_team_total_points,home_B1_point,home_B2_point,...,away_M4_point,away_T1_point,away_T2_point,away_K1_point,home_B1_value,home_B2_value,home_B3_value,tor_len,home_len,away_len
0,0.013057,3.615198e-07,3.615198e-06,0.000033,3.615198e-07,0.000728,0.000218,0.000237,0.000026,0.000026,...,0.000027,0.000029,0.000028,0.000029,0.506128,0.723040,0.469976,1.012256e-05,1.446079e-06,2.530639e-06
1,0.010816,5.988505e-07,1.736666e-05,0.000027,2.994252e-07,0.000603,0.000214,0.000158,0.000022,0.000022,...,0.000019,0.000022,0.000022,0.000020,0.598850,0.748563,0.284454,8.383906e-06,1.497126e-06,1.497126e-06
2,0.011751,9.758002e-07,3.252667e-07,0.000030,0.000000e+00,0.000655,0.000188,0.000196,0.000023,0.000023,...,0.000023,0.000025,0.000024,0.000023,0.813167,0.487900,0.317135,9.107468e-06,3.903201e-06,1.301067e-06
3,0.008028,8.886407e-07,5.554005e-06,0.000020,2.221602e-07,0.000447,0.000111,0.000151,0.000016,0.000016,...,0.000017,0.000019,0.000018,0.000018,0.333240,0.777561,0.533184,1.777281e-06,1.332961e-06,1.332961e-06
4,0.011754,9.758001e-07,3.252667e-06,0.000030,0.000000e+00,0.000655,0.000188,0.000213,0.000023,0.000023,...,0.000024,0.000026,0.000025,0.000026,0.813167,0.487900,0.317135,9.107468e-06,3.903201e-06,2.276867e-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2543,0.025228,4.857574e-05,9.143669e-06,0.000053,0.000000e+00,0.001156,0.000711,0.000763,0.000038,0.000038,...,0.000038,0.000041,0.000038,0.000038,0.514331,0.514331,0.685775,4.571834e-06,5.714793e-06,4.571834e-06
2544,0.067171,1.247654e-04,4.108128e-05,0.000140,0.000000e+00,0.003077,0.001463,0.002183,0.000093,0.000088,...,0.000114,0.000119,0.000114,0.000116,0.760764,0.456459,0.456459,1.217223e-05,1.065070e-05,1.065070e-05
2545,0.002878,2.607915e-07,2.216727e-06,0.000006,0.000000e+00,0.000132,0.000096,0.000092,0.000005,0.000005,...,0.000005,0.000005,0.000005,0.000005,0.619380,0.521583,0.586781,5.215829e-07,3.911872e-07,4.563851e-07
2546,0.004803,5.983142e-06,6.527064e-07,0.000010,0.000000e+00,0.000220,0.000155,0.000164,0.000008,0.000008,...,0.000008,0.000009,0.000008,0.000008,0.815883,0.380745,0.435138,8.702752e-07,8.702752e-07,5.439220e-07


In [ ]:
df_corr = df_temp.corr() # 각 열간의 상관계수 반환 

df_label과 df_temp(normalize된 값)

In [ ]:
y_es = df_label.to_numpy()
x_es = df_temp.to_numpy()
x_t, x_val , y_t, y_val = train_test_split(x_es, y_es, random_state=28)

In [ ]:
params = {
    "estimator__colsample_bylevel": uniform(0.7, 0.3),
    "estimator__gamma": uniform(0, 0.5),
    "estimator__learning_rate": uniform(0.003, 0.3),
    "estimator__max_depth": randint(2, 6),
    "estimator__n_estimators": randint(100, 500), 
    "estimator__subsample": uniform(0.6, 0.4)
}

In [ ]:
#Multioutput regression are regression problems that involve predicting two or more numerical values given an input example.
xgb_model = MultiOutputRegressor(xgb.XGBRegressor(objective='reg:squarederror', eval_metric=['mae'], random_state = 1, use_label_encoder=False))

In [ ]:
#search : 하이퍼 파라미터 튜닝
search = RandomizedSearchCV(xgb_model, param_distributions=params, random_state =1, n_iter=5, cv=3,
                            verbose=0, n_jobs=1, return_train_score=True) 
search.fit(x_t, y_t) 

best_param = search.best_params_
print('-')
print(search.best_score_)
print(best_param)
print('-')

-
0.07262575076506832
{'estimator__colsample_bylevel': 0.9147911547905092, 'estimator__gamma': 0.40137875196868245, 'estimator__learning_rate': 0.030840242592221373, 'estimator__max_depth': 3, 'estimator__n_estimators': 101, 'estimator__subsample': 0.9460081007915934}
-


In [ ]:
#RandomizedSearchCV로 찾은 best param을 모델에 적용
xgb_model = MultiOutputRegressor(xgb.XGBRegressor(objective='reg:squarederror',eval_metric=['mae'], random_state = 1, 
                              use_label_encoder=False, **best_param)).fit(x_t, y_t) 

In [ ]:
#MAE : MSE와 유사한 loss function
#이 코드에서 loss function이 적용되는 부분이 어디지? 못 찾았음
print('MAE: ',mean_absolute_error(y_val, xgb_model.predict(x_val)))

MAE:  0.8672534859676376


In [ ]:
#K-fold 교차 검증:
#전체 훈련 데이터를 K개의 그룹으로 묶고, 그중 한 그룹을 검증셋으로 사용합니다. 
#이때 각각의 모든 데이터 그룹이 적어도 한 번씩은 검증셋으로 사용되어야 합니다. 
#K-fold 교차 검증을 사용함으로써 검정 데이터에 모델이 과적합 되는 것을 막을 수 있습니다. 
folds = KFold(n_splits = 5, shuffle = True, random_state = 100)
scores = cross_val_score(xgb_model, x_val, y_val, scoring='neg_mean_absolute_error', cv=folds, verbose=0)
print('KFold_Neg_MAE: ',np.mean(scores))

KFold_Neg_MAE:  -0.9150680176862405


In [ ]:
#inp_list: 사용자가 입력한 home_team, away team
def predict_score(inp_list, autofill, Norm, seed, model):
    ind_list = []
    for i in inp_list:
        ind_list.append(nat_dic.get(i))
    seed[:,1] = ind_list[0]
    seed[:,2] = ind_list[1]
    x_test = Norm.fit_transform(seed) # Norm.fit_transform() 의 역할은 ? 
    pred = model.predict(x_test)
    return np.ravel(pred).tolist()

In [ ]:
#inp = str(input("Enter Home & Away team (like Home Away): "))
autofill = True
#inp = inp.split(' ')
inp = ['Jordan', 'Morocco']

# **mixed** : seed = np.delete를 주석처리하면 해결됨. 기존 코드 체크하기

In [ ]:
x_train.shape[0]

2548

In [ ]:
def create_params():
  seed = x_train[random.randint(0,x_train.shape[0])].reshape(1,-1)
  # random.randin() 함수에서는 0부터 len(x_train) 까지해서 하나를 뽑고 있음 
  # 
  #seed = np.delete(seed, 4, axis=1) -> 기존 배열에서 5번째 열 제거 ? Delete 함수 자체는 flatten 한다 -> 우리는 이미 1차원 array로 만들었으니까 상관 ㄴㄴ
  # array([[ 1,  2,  3,  4],
#        [ 5,  6,  7,  8],
#        [ 9, 10, 11, 12]]) 이면 array([ 1,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12])
  params = {
    'autofill': True,
    'Norm': Norm,
    'seed': seed,
    'model': xgb_model
  }
  return params

In [ ]:
df_fifa = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/fifa/fifa_dataset.csv")

In [ ]:
df_example = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/fifa/results.csv")

In [ ]:
df_fifa.loc[df_fifa["home_team"] == "Portugal"].shape[0]

49

In [ ]:
df_example.loc[df_example["home_team"] == "South Korea"].shape[0]

499

In [ ]:
df_example.loc[df_example["home_team"] == "United States"].shape[0]

445

In [ ]:

#예선 팀 조 편성 결과 입력
Group_A= ["Netherlands","Senegal","Ecuador","Qatar"]
Group_B= ["England","United States","Iran","Wales"] # United States 가 없음 ;; 
Group_C= ["Argentina","Poland","Mexico", "Saudi Arabia"]
Group_D= ["France","Australia","Tunisia","Denmark"]
Group_E= ["Japan","Spain","Germany","Costa Rica"]
Group_F= ["Morocco","Croatia","Belgium","Canada"]
Group_G= ["Brazil","Switzerland","Cameroon","Serbia"]
Group_H= ["Portugal","South Korea","Uruguay","Ghana"] # South Korea 가 없음 ;; 
Groups={"Group A":Group_A,"Group B":Group_B,"Group C":Group_C,"Group D":Group_D,"Group E":Group_E,"Group F":Group_F,"Group G":Group_G,"Group H":Group_H}


In [ ]:
def create_input(home_team, away_team):
  #인풋으로 home_team, away_team만 주면 되는가? 아니면 별도의 벡터 생성 필요?
  ind_list = [home_team, away_team]
  return ind_list#input_vector

In [ ]:
def select_winning_team(ans, qual): #probability_array
    pred_ans = copy.deepcopy(ans)
    #예선 : 소수점 x, 정수로 승패 판결
    if qual == True :
      #정수
      pred_ans[0] = round(pred_ans[0])
      pred_ans[1] = round(pred_ans[1])
      
    #본선
    else :
      #소수점 1자리
      pred_ans[0] = round(pred_ans[0], 1)
      pred_ans[1] = round(pred_ans[1], 1)
    
    if (pred_ans[0]>pred_ans[1]):
          out=0 # home이 우승
    elif (pred_ans[0]<pred_ans[1]):
          out=1 # away가 우승
    elif (pred_ans[0]==pred_ans[1]):
          out=2 #비김

    return out, pred_ans

In [ ]:
#inp_list: 사용자가 입력한 home_team, away team 
# predict_score(input, **params) 
def predict_score(inp_list, autofill, Norm, seed, model):
    ind_list = [] # team1, team2 
    for i in inp_list:
        ind_list.append(nat_dic.get(i))
    #seed의 목적이 뭐지? : ind_list , [141, 22]
    seed[:,1] = ind_list[0] # seed[:, 1] 행에서 두번째 열의 정보를 ind_list[0] 값으로 바꾸기 (세로)
    seed[:,2] = ind_list[1] # 행에서 세번째 열의 정보를  ind_list[1]
    x_test = Norm.fit_transform(seed)
    pred = model.predict(x_test)
    return np.ravel(pred).tolist()

In [ ]:

##Group stage Matches
print("========Qualifying Games=========")
Group_standings={}
for grp_name in list(Groups.keys()):
    print(f"[{grp_name} Matches]")
    #해당 그룹의 팀들
    probable_countries=Groups[grp_name]

    team_wins_dct={}
    goal_scored_dct={}
    goal_against_dct={}
    win_dct={}
    draw_dct={}
    lost_dct={}
    
    for i in range(len(probable_countries)):
        #team i: team1, team j : team2
        j=i+1
        team_1=probable_countries[i]

        team_wins=0
        
        while j<len((probable_countries)):
            team_2=probable_countries[j]
            team_lst=[team_1,team_2]

            #인풋으로 home_team, away_team만 주면 되는가? 아니면 별도의 벡터 생성 필요?
            input=create_input(team_1, team_2)#np.array([[year,stadium_num,team_1_num,team_2_num]])
            params = create_params()
            #ans : 소수점 ex 1.3 : 1.5
            ans = predict_score(input, **params)

            #qual : 예선이면 true, 본선이면 false
            qual = True
           
            win, prob_lst=select_winning_team(ans, qual)
            print("applied_score : ", prob_lst,", predicted_score: ", ans)
            
            #본인 팀 스코어
            goal_scored_dct[team_1] = goal_scored_dct.get(team_1,0)+prob_lst[0]
            goal_scored_dct[team_2] = goal_scored_dct.get(team_2,0)+prob_lst[1]
            #상대 팀 스코어
            goal_against_dct[team_1] = goal_against_dct.get(team_1,0)+prob_lst[1]
            goal_against_dct[team_2] = goal_against_dct.get(team_2,0)+prob_lst[0]

            try:
                print(f" {team_1} vs {team_2} \n  {team_lst[win]} wins 🎊🏆⚽🎖️\n")
                #team1 우승
                if (win)==0:
                    team_wins_dct[team_1] = team_wins_dct.get(team_1,0)+2
                    team_wins_dct[team_2] = team_wins_dct.get(team_2,0)
                    #team1 1승 추가
                    win_dct[team_1] = win_dct.get(team_1,0)+1
                    win_dct[team_2] = win_dct.get(team_2,0)
                    #team2 1패 추가
                    lost_dct[team_2] = lost_dct.get(team_2,0)+1
                    lost_dct[team_1] = lost_dct.get(team_1,0)
                    #비긴 팀 없음
                    draw_dct[team_2] = draw_dct.get(team_2,0)
                    draw_dct[team_1] = draw_dct.get(team_1,0)

                elif (win)==1:
                    team_wins_dct[team_2] = team_wins_dct.get(team_2,0)+2
                    team_wins_dct[team_1] = team_wins_dct.get(team_1,0)
                    #team2 1승 추가
                    win_dct[team_2] = win_dct.get(team_2,0)+1
                    win_dct[team_1] = win_dct.get(team_1,0)
                    #team1 1패 추가
                    lost_dct[team_1] = lost_dct.get(team_1,0)+1
                    lost_dct[team_2] = lost_dct.get(team_2,0)
                    #비긴 팀 없음
                    draw_dct[team_1] = draw_dct.get(team_1,0)
                    draw_dct[team_2] = draw_dct.get(team_2,0)

            except IndexError:
              #비긴 경우
              print(f"{team_1} vs {team_2} \n  Match Draw ⚽⚽⚽\n") 
              team_wins_dct[team_1] = team_wins_dct.get(team_1,0)+1
              team_wins_dct[team_2] = team_wins_dct.get(team_2,0)+1
              
              #비김
              draw_dct[team_1] = draw_dct.get(team_1,0)+1
              draw_dct[team_2] = draw_dct.get(team_2,0)+1
              #승패 없음
              win_dct[team_1] = win_dct.get(team_1,0)
              lost_dct[team_1] = lost_dct.get(team_1,0)
              #승패 없음
              win_dct[team_2] = win_dct.get(team_2,0)
              lost_dct[team_2] = lost_dct.get(team_2,0)


            j=j+1
    
    group_results=[win_dct,draw_dct,lost_dct,team_wins_dct,goal_scored_dct,goal_against_dct]
    Group_standings[grp_name]=group_results


========Qualifying Games=========
[Group A Matches]
applied_score :  [2, 1] , predicted_score:  [1.793779969215393, 0.609484076499939]
 Netherlands vs Senegal 
  Netherlands wins 🎊🏆⚽🎖️

applied_score :  [1, 2] , predicted_score:  [1.0448805093765259, 2.4252264499664307]
 Netherlands vs Ecuador 
  Ecuador wins 🎊🏆⚽🎖️

applied_score :  [1, 1] , predicted_score:  [1.2987449169158936, 0.9554417133331299]
Netherlands vs Qatar 
  Match Draw ⚽⚽⚽

applied_score :  [1, 2] , predicted_score:  [1.0328636169433594, 1.8157397508621216]
 Senegal vs Ecuador 
  Ecuador wins 🎊🏆⚽🎖️

applied_score :  [2, 1] , predicted_score:  [1.7660404443740845, 0.8628866672515869]
 Senegal vs Qatar 
  Senegal wins 🎊🏆⚽🎖️

applied_score :  [1, 1] , predicted_score:  [1.2171123027801514, 1.074722170829773]
Ecuador vs Qatar 
  Match Draw ⚽⚽⚽

[Group B Matches]
applied_score :  [1, 1] , predicted_score:  [1.1755342483520508, 0.9338874816894531]
England vs United States 
  Match Draw ⚽⚽⚽

applied_score :  [1, 1] , predicted_

4 없애기 : 어떤 요소 없앴었다. 왜 없앴더라? 왜 기존 코드에선 없애도 되고 우리 코드에선 안되지?

**random 으로 seed를 왜 추출하는가?**